## Cost analysis of Azure resources using LLM

Download cost analysis report. \
Ensure you've set the right filters while downloading so that the CSV contains required fields.

In [11]:
import pandas as pd
df = pd.read_csv('cost-analysis.csv')
df.head()

,UsageDate,ResourceId,ResourceType,ResourceLocation,ResourceGroupName,ServiceName,ServiceTier,Meter,CostUSD,Cost,Currency
0,2023-08-01,/subscriptions/bd798494-32ba-4eb2-8e27-c6852eb...,microsoft.databricks/workspaces,EU North,adb-poc-rg,Azure Databricks,NaN,Premium All-purpose Compute DBU,3.749258,307.211937,INR
1,2023-08-01,/subscriptions/bd798494-32ba-4eb2-8e27-c6852eb...,microsoft.databricks/workspaces,EU North,adb-poc-rg,Azure Databricks,NaN,Premium All-Purpose Photon DBU,0.359120,29.426039,INR
2,2023-08-01,/subscriptions/bd798494-32ba-4eb2-8e27-c6852eb...,microsoft.databricks/workspaces,EU North,adb-poc-rg,Azure Databricks,NaN,Premium All-Purpose Photon DBU,4.867577,398.846326,INR
3,2023-08-01,/subscriptions/bd798494-32ba-4eb2-8e27-c6852eb...,microsoft.keyvault/vaults,EU North,adb-poc-rg,Key Vault,NaN,Operations,0.000121,0.009895,INR
4,2023-08-01,/subscriptions/bd798494-32ba-4eb2-8e27-c6852eb...,microsoft.keyvault/vaults,EU North,adb-poc-rg,Key Vault,NaN,Operations,0.000040,0.003298,INR


Import packages and load environment variables

In [1]:
import os
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv 
from langchain.agents.agent_types import AgentType
from langchain.agents import create_csv_agent

load_dotenv()

/Users/rohannevrikar/Desktop/repos/azure-openai-demo/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True

Setup Azure OpenAI LLM model

In [2]:
llm = AzureChatOpenAI(
    deployment_name=os.getenv("AZURE_DEPLOYMENT_NAME"),
    model_name="gpt-35-turbo",
    openai_api_base=os.getenv("AZURE_OPENAI_ENDPOINT"), 
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_type="azure"
)

Create Langchain's out of the box CSV agent which helps us interact with a CSV. \
Ref- https://python.langchain.com/docs/integrations/toolkits/csv

In [3]:
agent = create_csv_agent(
    llm,
    "cost-analysis.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,    
)

CSV agent chains dataframe and Python agent, which basically generates Python code using LLM and runs it for the data frame.

In [4]:
agent("Which was the most expensive service in the month of August?")



> Entering new AgentExecutor chain...
Thought: We need to filter the dataframe to only include entries from August, group the entries by ServiceName, and then sum the CostUSD for each group. Finally, we can find the ServiceName with the highest total cost.
Action: python_repl_ast
Action Input:
```
# first filter dataframe to only include entries from August
august_df = df[df['UsageDate'].dt.month == 8]

# group by ServiceName and sum the CostUSD for each group
grouped_df = august_df.groupby('ServiceName')['CostUSD'].sum()

# find the ServiceName with the highest total cost
most_expensive_service = grouped_df.idxmax()
```
Observation: AttributeError: Can only use .dt accessor with datetimelike values
Thought:The UsageDate column is not in a datetime format. We need to convert it before filtering by month.
Action: python_repl_ast
Action Input:
```
# convert UsageDate column to datetime format
df['UsageDate'] = pd.to_datetime(df['UsageDate'])

# filter dataframe to only include entries 

{'input': 'Which was the most expensive service in the month of August?',
 'output': 'The most expensive service in the month of August was Azure Databricks.'}